In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.display import display
from sklearn.model_selection import train_test_split

from warnings import filterwarnings
filterwarnings('ignore')

In [35]:
def identity(x): 
    unique_list = list(np.unique(x))
    if len(unique_list)==1:
        return unique_list[0]
    else:
        return unique_list
# standard = pd.read_excel('[MOA_CDM]variable_selection_by_methods.xlsx',engine='openpyxl',sheet_name='concept_set_by_methods')
# standard = pd.read_csv('MOA_CDM_variable_selection_by_methods.csv')
standard = pd.read_csv('selected_vars_all.csv')

In [3]:
# origin
def load_df_and_concept_id(outcome_name,method):
    # 수정
    importsql_output_dir = 'data/19000101/importsql/{}'.format(outcome_name)
    
    meas_df = pd.read_csv('{}/{}_meas_df.txt'.format(importsql_output_dir, outcome_name), low_memory=False)
    drug_df = pd.read_csv('{}/{}_drug_df.txt'.format(importsql_output_dir, outcome_name), low_memory=False)
    proc_df = pd.read_csv('{}/{}_proc_df.txt'.format(importsql_output_dir, outcome_name), low_memory=False)
    cond_df = pd.read_csv('{}/{}_cond_df.txt'.format(importsql_output_dir, outcome_name), low_memory=False)
    proc_df['concept_value'] = 1
    cond_df['concept_value'] = 1
    drug_df['concept_value'] = 1
    
    cond1 = standard['outcome_name']==outcome_name
    cond2 = standard['method']==method
    concept_ids_cond = standard.loc[cond1&cond2,'cond'].values[0]
    concept_ids_drug = standard.loc[cond1&cond2,'drug'].values[0]
    concept_ids_proc = standard.loc[cond1&cond2,'proc'].values[0]
    concept_ids_meas = standard.loc[cond1&cond2,'meas'].values[0]
    
    concept_id_cond_list = list(eval(concept_ids_cond))
    concept_id_drug_list = list(eval(concept_ids_drug))
    concept_id_proc_list = list(eval(concept_ids_proc))
    concept_id_meas_list = list(eval(concept_ids_meas))
    return {'meas_df':meas_df,'drug_df':drug_df,'proc_df':proc_df,'cond_df':cond_df,
           'concept_id_cond_list':concept_id_cond_list,'concept_id_drug_list':concept_id_drug_list,
           'concept_id_proc_list':concept_id_proc_list,'concept_id_meas_list':concept_id_meas_list}

In [4]:
def load_df_and_concept_id2(outcome_name,method):
    # 수정
    importsql_output_dir = 'data/19000101/importsql/{}/all_domain_df.txt'.format(outcome_name)
    df = pd.read_csv(importsql_output_dir)
#     path = f'../Hospitals/{insti}/220927/importsql/{outcome_name}/all_domain_df.txt'
#     df = pd.read_csv(path) 
    
    meas_df = df.query('concept_domain=="meas"')
    drug_df = df.query('concept_domain=="drug"')
    proc_df = df.query('concept_domain=="proc"')
    cond_df = df.query('concept_domain=="cond"')
    
    proc_df.loc[:,'concept_value'] = 1
    cond_df.loc[:,'concept_value'] = 1
    drug_df.loc[:,'concept_value'] = 1
    
    cond1 = standard['outcome_name']==outcome_name
    cond2 = standard['method']==method
    concept_ids_cond = standard.loc[cond1&cond2,'cond'].values[0]
    concept_ids_drug = standard.loc[cond1&cond2,'drug'].values[0]
    concept_ids_proc = standard.loc[cond1&cond2,'proc'].values[0]
    concept_ids_meas = standard.loc[cond1&cond2,'meas'].values[0]
    
    concept_id_cond_list = list(eval(concept_ids_cond))
    concept_id_drug_list = list(eval(concept_ids_drug))
    concept_id_proc_list = list(eval(concept_ids_proc))
    concept_id_meas_list = list(eval(concept_ids_meas))
    return {'meas_df':meas_df,'drug_df':drug_df,'proc_df':proc_df,'cond_df':cond_df,
           'concept_id_cond_list':concept_id_cond_list,'concept_id_drug_list':concept_id_drug_list,
           'concept_id_proc_list':concept_id_proc_list,'concept_id_meas_list':concept_id_meas_list}

In [5]:
# concept_id_dict = dict(zip(df['concept_id'],df['concept_name']))
def pivot_and_concat(df, concept_id_list):
    timedelta = pd.to_datetime(df['cohort_start_date']) - pd.to_datetime(df['concept_date'])
    cond1 = timedelta.dt.days >= 0
    cond2 = timedelta.dt.days <= 30
    df_1month = df[cond1&cond2]

    df_1month_grouped = df_1month.groupby(['person_id','concept_id']).agg({'cohort_start_date':identity,'age':identity,
                                                                           'sex':identity,'label':identity,'concept_domain':identity,'concept_value':'mean'})

    no_pivoted = df_1month_grouped.reset_index(level=0).drop(['concept_value'],axis=1).drop_duplicates().set_index('person_id')
    pivoted = df_1month_grouped.reset_index().pivot(index='person_id', columns = 'concept_id',values='concept_value')
    # 수정(할 필요는 딱히 없을듯)
    pivoted_selected_vars = pivoted[np.intersect1d(pivoted.columns,list(map(int,concept_id_list)))]

    df_pivoted = pd.concat([no_pivoted, pivoted_selected_vars],axis=1)
    df_pivoted = df_pivoted.reset_index()
    return df_pivoted

In [6]:
def merge_concat(df1, df2, df3, df4, on=['person_id','cohort_start_date','age','sex','label']):
    pivots = [df1,df2,df3,df4]
    pivots_order = sorted(pivots,key=lambda x:x.shape[0])
    if set(pivots_order[0]['person_id'].values).issubset(set(pivots_order[3]['person_id'])):
        print('the biggest set contains the smallest set')
    if set(pivots_order[1]['person_id'].values).issubset(set(pivots_order[3]['person_id'])):
        print('the biggest set contains the second smallest set')
    if set(pivots_order[2]['person_id'].values).issubset(set(pivots_order[3]['person_id'])):
        print('the biggest set contains the third smallest set')
    outcome_information = pd.merge(pivots_order[-1].drop('concept_domain',axis=1), 
                               pivots_order[-2].drop('concept_domain',axis=1),
                               how='outer',on=on)
    outcome_information = pd.merge(outcome_information, 
                               pivots_order[-3].drop('concept_domain',axis=1),
                               how='outer',on=on)
    outcome_information = pd.merge(outcome_information, 
                               pivots_order[-4].drop('concept_domain',axis=1),
                               how='outer',on=on)
    return outcome_information

In [7]:
# measure는 median, 나머지는 0으로 결측치 보간
def Impute(df,outcome_dict):
    drug_concept = outcome_dict['concept_id_drug_list']
    cond_concept = outcome_dict['concept_id_cond_list']
    proc_concept = outcome_dict['concept_id_proc_list']
    meas_concept = outcome_dict['concept_id_meas_list']

    impute_median_part = meas_concept
    impute_zero_part = drug_concept+cond_concept+proc_concept
    
    impute_median_part = np.intersect1d(df.columns.tolist(),list(map(int,impute_median_part))).astype(int)
    impute_zero_part = np.intersect1d(df.columns.tolist(),list(map(int,impute_zero_part))).astype(int)
    df[impute_median_part] = df[impute_median_part].fillna(df[impute_median_part].median())
    df[impute_zero_part] = df[impute_zero_part].fillna(0)
    
    return df

In [8]:
def make_table(outcome_name, method):
    outcome_dict = load_df_and_concept_id(outcome_name, method)
    
    drug_pivoted = pivot_and_concat(outcome_dict['drug_df'],outcome_dict['concept_id_drug_list'])
    cond_pivoted = pivot_and_concat(outcome_dict['cond_df'],outcome_dict['concept_id_cond_list'])
    proc_pivoted = pivot_and_concat(outcome_dict['proc_df'],outcome_dict['concept_id_proc_list'])
    meas_pivoted = pivot_and_concat(outcome_dict['meas_df'],outcome_dict['concept_id_meas_list'])
    
    outcome_info = merge_concat(drug_pivoted,cond_pivoted,proc_pivoted,meas_pivoted)
    
    outcome_info = Impute(outcome_info,outcome_dict)
    
    return outcome_info

In [ ]:
columns = dict()
for outcome_name in ['Candesartan','Losartan','Olmesartan','Valsartan','Irbesartan',
                    'Celecoxib','Acetaminophen','Celecoxib_2','Vancomycin','Naproxen']:
    
    # 보내드린 폴더의 stage1/input 경로에 맞춰주시면 됩니다.
    save_path_train = f'stage1/input/{outcome_name}_{method}_train.csv'
    save_path_val = f'stage1/input/{outcome_name}_{method}_val.csv'
    df = make_table(outcome_name,method)

    columns[outcome_name] = df.columns.tolist()

    train, val = train_test_split(df, stratify=df['label'], test_size=0.3, random_state=123)
    
    train.to_csv(save_path_train,index=False)
    val.to_csv(save_path_val,index=False)

In [22]:
import pickle
with open('columns.pickle','wb') as f:
    pickle.dump(columns, f)

In [23]:
import pickle
with open('columns.pickle','rb') as f:
    konyang_col = pickle.load(f)

In [47]:
use_col = dict()
for drug,k_col in konyang_col.items():
    
    gangnam_df = pd.read_csv(f'../2022_WIM_test/stage1/all_drugs_propensity_matching/gangnam/{drug}_gangnam_ExtraTrees_train.csv')
    sinchon_df = pd.read_csv(f'../2022_WIM_test/stage1/all_drugs_propensity_matching/sinchon/{drug}_sinchon_ExtraTrees_train.csv')
    g_col = gangnam_df.columns.tolist()
    s_col = sinchon_df.columns.tolist()
    
    
    gs_col = np.intersect1d(g_col, s_col)
    gsk_col = np.intersect1d(gs_col, k_col)

    print(len(g_col), len(s_col),' gs ', len(gs_col),' k ', len(k_col),' gsk ', len(gsk_col))
    use_col[drug] = list(gsk_col)
    
#     print(drug,' ' ,len(k_col))
#     cond1 = standard['outcome_name']==drug
#     cond2 = standard['method'] == 'ExtraTrees'
    
    
    
#     temp = 0
#     for col in ['common','cond','drug','meas','proc']:
#         temp += len(eval(standard[cond1&cond2][col].values[0]))
#     print(drug, ' ', temp)


205 205  gs  205  k  71  gsk  71
211 211  gs  211  k  74  gsk  74
114 114  gs  114  k  52  gsk  51
232 232  gs  232  k  73  gsk  73
99 99  gs  99  k  46  gsk  46
177 177  gs  177  k  62  gsk  61
363 363  gs  363  k  122  gsk  122
189 189  gs  189  k  77  gsk  75
424 424  gs  424  k  132  gsk  132
317 317  gs  317  k  85  gsk  84


In [54]:
pd.Series(use_col).rename('common_columns').to_csv('common_columns.csv')